# Workshop: Introducción a Generative AI en Oracle y Creación de Agentes con LangChain

Bienvenidos al workshop. En esta sesión vamos a explorar cómo usar los **servicios de IA generativa de Oracle** para resolver problemas reales y luego **crear un agente inteligente** usando **LangChain** que pueda interactuar con estos servicios.

## Objetivos de la sesión
- Conocer la oferta de **Oracle Cloud Infrastructure (OCI Generative AI)** y cómo integrarla desde Python.
- Ejecutar peticiones a modelos de lenguaje para **generar texto** de manera controlada.
- Construir un **agente con LangChain** que use herramientas (como SQL o RAG) para responder preguntas de forma autónoma.
- Aprender buenas prácticas para **orquestar flujos de trabajo** y extender capacidades de los modelos.

## Requisitos previos
- Conocimientos básicos de Python 🐍
- Tener acceso a una cuenta de **Oracle Cloud** con permisos para usar **OCI Generative AI**
- Familiaridad básica con entornos virtuales y Jupyter Notebooks.

> 💡 **Tip:** este notebook está diseñado para ser práctico y paso a paso. Podrás copiar, ejecutar y modificar el código para experimentar con los conceptos que vamos a explicar.

¡Vamos a empezar!

## A continuación... 

📰 Recopilaremos noticias sobre el paro del 16 de Septiembre ocurrido en la ciudad de Bogotá 

🤖 Consumiremos un modelo de lenguaje alojado en Oracle Cloud 

🔍 Construiremos un agente con langchain que es capaz de responder a preguntas relacionadas con el paro del 16 de septiembre 

## Instalación

In [ ]:
!pip install tavily-python
!pip install -U langchain-community langchain-core oci

In [ ]:
import requests
from langchain_core.tools import tool
from typing import Dict, List
import re
from tavily import TavilyClient
import oci
import json
from oci.auth.signers import get_resource_principals_signer
from oci.config import from_file
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOCIGenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from IPython.display import Markdown, display
import json


Nuestro agente necesita acceso a la web para acceder a las últimas noticias del tema que le hemos indicado, para esto, es necesario configurar una herramienta que le permita a nuestro agente navegar por portales de noticias, por eso usaremos [Tavily](https://www.tavily.com/).

## 🪪 Registro en Tavily (paso a paso)

1) Abre **https://app.tavily.com/home** y haz clic en **Sign Up**. Verifica tu correo electrónico para activar la cuenta. ![image1](./images/tavily_signup.png)
2) Inicia sesión: en la **página principal** verás tu **API Key**. Haz **Copy** para copiarla. ![image2](./images/api_key.png)


In [ ]:
# Pega la API Key de Tavily aquí
TAVILY_API_KEY = "pega_aqui_tu_api_key"

In [ ]:
assert TAVILY_API_KEY != "pega_aqui_tu_api_key", "Por favor, pega tu API Key de Tavily en la variable TAVILY_API_KEY"

In [ ]:
COLOMBIA_PARO_DOMAINS = [
    "bogota.gov.co", "larepublica.co", "eltiempo.com", "semana.com", "noticiascaracol.com", "elcolombiano.com", "rcnradio.com", "elespectador.com", "las2orillas.co", "bluradio.com", "ambientarteradio.com", "elnuevosiglo.com.co", "pulzo.com", "lasillavacia.com"
]
@tool
def get_paro_comprehensive_news(pregunta:str="Horarios, agenda, causas y afectaciones del paro de transporte de Septiembre en Bogotá") -> str:
    """Devuelve info/noticias de horarios, agenda, causas y afectaciones del paro de transporte de Septiembre en Bogotá."""

    client = TavilyClient(TAVILY_API_KEY)
    response = client.search(
    query=pregunta,
    #include_domains=CHILE_FIESTAS_DOMAINS,
    #topic="news",
    #days=45,
    #max_results=10
    )
    return response

## Configuración de la autenticación del SDK de OCI

Desde este notebook es necesario acceder a algunos servicios de Oracle, como el servicio de Generative AI, aunque ejecutes este notebook en cloud o de forma local, es necesario configurar las credenciales en la máquina que realiza el consumo del servicio. 

```
En los pasos anteriores fue necesario descargar un archivo terminado en .pem y copiar una configuración con el siguiente estilo
[DEFAULT]
user=ocid1.user.oc1..
fingerprint=95:e1:09
tenancy=ocid1.tenancy.oc1..
region= 
```

A continuación usaremos esos objetos.

In [26]:
# crea carpeta y permisos
!mkdir -p /home/datascience/.oci

mkdir: /home/datascience: Operation not supported


In [ ]:
# Ver tu HOME y listar (incluye ocultos)
!echo $HOME
!ls -la $HOME | head -n 30

In [ ]:
!mkdir -p ~/.oci
!ls -la ~/.oci


In [ ]:
!mv ~/oci_api_key.pem ~/.oci/oci_api_key.pem
!chmod 600 ~/.oci/oci_api_key.pem
!ls -la ~/.oci

A continuación, crearemos el archivo de configuración en la ruta ~/.oci/config, vamos a copiar los valores de la configuración mostrada en pantalla y a reemplazarlos en la siguiente línea.

Reemplazaremos _ocid1.user.oc1.._ por el ocid del usuario mostrado en pantalla
Reemplazaremos _fingerprint_ por el figerprint mostrado en pantalla
Reemplazaremos _fingerprint_ por el figerprint mostrado en pantalla
Reemplazaremos _fingerprint_ por el figerprint mostrado en pantalla
🚨 No reemplazaremos _key_file_ por ninguna ruta si estamos ejecutando este notebook en DataScience. Si queremos ejecutar este notebook de forma local, podemos reemplazar la ruta por ~/.oci/nombre_de_la_key.pem 

In [ ]:
%%bash
cat > ~/.oci/config <<'CFG'
[DEFAULT]
user=ocid1.user.oc1..
fingerprint=95:e1:09
tenancy=ocid1.tenancy.oc1..
region=
key_file=/home/datascience/.oci/oci_api_key.pem
CFG

echo "Config creado en ~/.oci/config"
cat ~/.oci/config | sed 's/fingerprint=.*/fingerprint=<oculto>/'

In [ ]:
# Quita posibles finales de línea de Windows (CRLF)
!sed -i 's/\r$//' ~/.oci/config

# mostrar
!sed -n '1,200p' ~/.oci/config

In [ ]:
config = from_file()

In [ ]:
# Descomenta únicamente la línea que corresponda a tu región
#REGION = "sa-saopaulo-1"
#REGION = "us-chicago-1"
#REGION = "uk-london-1"
#REGION = "eu-frankfurt-1"
#REGION = "ap-osaka-1"

In [ ]:
assert REGION in ["sa-saopaulo-1", "us-chicago-1", "uk-london-1", "eu-frankfurt-1", "ap-osaka-1"], "Por favor, descomenta la línea que corresponda a tu región"

In [ ]:
# Aquí debes pegar el OCID de tu compartimento
# Encuentra el OCID de tu compartimento en la consola de Oracle Cloud, en la sección de Compartments https://cloud.oracle.com/identity/compartments
COMPARTMENT_ID = "ocid1.compartment.oc1...."

In [ ]:
SERVICE_ENDPOINT = f"https://inference.generativeai.{REGION}.oci.oraclecloud.com"
MODEL_ID = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceya6dvgvvj3ovy4lerdl6fvx525x3yweacnrgn4ryfwwcoq"

In [ ]:
if MODEL_ID is None:
    from oci.generative_ai import GenerativeAiClient
    genai = GenerativeAiClient()
    models = genai.list_models(
        compartment_id=COMPARTMENT_ID,
        capability=["CHAT"],
        lifecycle_state="ACTIVE"
    ).data.items
    assert models, "No hay modelos CHAT visibles en el compartimento. Revisa permisos/compartimento."
    MODEL_ID = models[0].id
    print("Usando modelo:", MODEL_ID)

# === Cliente de inferencia ===
inf = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config,
    service_endpoint=SERVICE_ENDPOINT
)

# === Prompt del usuario ===
user_input = "hola, tienes información del paro del 16 de Septiembre en Bogotá?"

# --- Construcción del request ---
content = oci.generative_ai_inference.models.TextContent(text=user_input)
message = oci.generative_ai_inference.models.Message(role="USER", content=[content])

chat_request = oci.generative_ai_inference.models.GenericChatRequest(
    api_format=oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC,
    messages=[message],
    max_tokens=600,
    temperature=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    top_p=0.75,
)

chat_detail = oci.generative_ai_inference.models.ChatDetails(
    serving_mode=oci.generative_ai_inference.models.OnDemandServingMode(model_id=MODEL_ID),
    chat_request=chat_request,
    compartment_id=COMPARTMENT_ID,
)

# === Llamada ===
resp = inf.chat(chat_detail)

# === Resultado ===
choices = resp.data.chat_response.choices
response_text = choices[0].message.content[0].text if choices else "No se generó respuesta."
print(json.dumps({"response": response_text}, indent=2, ensure_ascii=False))

## 🤖 Creación del Agente LangChain

In [ ]:
# Configura tu endpoint y compartimento
ENDPOINT = f"https://inference.generativeai.{REGION}.oci.oraclecloud.com"

llm = ChatOCIGenAI(
  model_id="meta.llama-4-maverick-17b-128e-instruct-fp8",
  service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
  compartment_id=COMPARTMENT_ID,
  provider="meta",
  model_kwargs={
    "temperature": 0.3, 
    "max_tokens": 800,   
    "top_p": 0.8,  
    "frequency_penalty": 0,
    "presence_penalty": 0,
  },
  auth_type="API_KEY",
  auth_profile="DEFAULT"
)

tools = [get_paro_comprehensive_news]

react_prompt_template = """Eres un asistente especializado en el paro de transporte del 16 de septiembre en Bogotá.
Responde en español de Colombia y usa las herramientas disponibles cuando sea útil.

REGLAS IMPORTANTES:

Tras usar una herramienta, resume con tus palabras los hallazgos (no pegues el JSON).

Prioriza información reciente y oficial (Alcaldía de Bogotá, Secretaría de Movilidad, medios reconocidos).

Incluye rutas afectadas, alternativas de transporte, horarios y enlaces útiles cuando existan.

Si hay discrepancias entre fuentes, indícalas brevemente.

Herramientas disponibles:
{tools}

Usa EXACTAMENTE este formato:

Question: la pregunta a responder
Thought: explica qué harás
Action: una de [{tool_names}]
Action Input: el input para la acción (o "" si no aplica)
Observation: resultado de la acción
Thought: analiza y sintetiza
Final Answer: respuesta clara y útil en español de Colombia

Comienza.

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(react_prompt_template)
agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=6,
    stream_runnable=False,
    handle_parsing_errors=True
)
pregunta = "cuál es la causa del paro?"
respuesta = agent_executor.invoke({"input": pregunta})

pregunta = "¿Qué organización participa en el paro?"
respuesta = agent_executor.invoke({"input": pregunta})


In [ ]:
respuesta

In [ ]:
# 1) Toma la salida del agente y conviértela a texto de forma segura
def _to_text(x):
    if isinstance(x, dict):
        # LangChain AgentExecutor suele devolver {"output": "..."}
        return x.get("output") or json.dumps(x, ensure_ascii=False, indent=2)
    return str(x)

insumos = _to_text(respuesta)   # <--- usa la variable 'respuesta' que ya tienes del agente

# 2) Prompt de análisis/síntesis para Fiestas Patrias (Chile)
analysis_prompt = f"""
Eres un analista colombiano especializado en **el paro de transporte del 16 de septiembre en Bogotá**.  
Usa solo los datos entregados más abajo para responder en **español de Colombia**, claro y útil.

**DATOS RECOLECTADOS** (pueden incluir JSON):  
{insumos}

---

### TAREAS  
1. Responde directamente a la pregunta: **"{pregunta}"**.  
2. Extrae y organiza lo más importante (si está disponible):  
   - **hora de inicio y fin**  
   - **zonas afectadas**  
   - **rutas de TransMilenio y SITP impactadas**  
   - **bloqueos o marchas**  
   - **alternativas de transporte**  
   - **enlaces oficiales**  
3. Indica si se reportan **medidas de la Alcaldía** (desvíos, refuerzos de policía, pico y placa solidario, teletrabajo sugerido, etc.) y cualquier impacto en **tráfico, colegios, oficinas y eventos** si está en los datos.  
4. Si hay discrepancias entre fuentes, menciónalas brevemente.  

---

### FORMATO (Markdown)

#### ## Resumen  
- 3–5 líneas con lo esencial (hora, zonas críticas, recomendaciones).  

#### ## Rutas y afectaciones (si hay datos)  
- **Sistema/sector – Afectación**: detalle de bloqueo o desvío | horario | recomendaciones | enlace  

## Recomendaciones (si hay datos)
- Lista de recomendaciones

#### ## Fuentes  
- Lista de URLs citadas (solo si aparecen en los datos).  

---

### REGLAS  
- **No inventes** datos ni enlaces. Si algo no está en los datos, escribe: *No disponible*.  
- Prioriza información **reciente y oficial** (Alcaldía de Bogotá, Secretaría de Movilidad, TransMilenio, medios confiables).
"""

# 3) Invoca la LLM (no streaming) y muestra en Markdown
analysis_response = llm.invoke(analysis_prompt)
display(Markdown(analysis_response.content))